# Retrieval Augmented Generation With Postgres, Ollama and LangChain

The notebook demonstrates how to use the retrieval-augmented generation (RAG) technique to make data stored in Postgres available to an Ollama LLM.

**0. Prerequsites**

* Install python and pip
* Start a Postgres container with pgvector and upload the movies dataset following the instructions from [chapter8.md](../chapter8.md)
* Start an Ollama container and download required embedding and large language models follow the [ai_samples/README.md](README.md) instructions.

**1. Install Required Modules**

In [ ]:
pip install -q psycopg2-binary==2.9.9 langchain-ollama==0.2.3

**2. Import Modules**

In [151]:
from langchain_ollama import OllamaLLM
import psycopg2

**3. Define Function Using Ollama LLM**

In [152]:
def answer_question(question, context):
    llm = OllamaLLM(model="tinyllama", temperature=0.6)

    prompt =""
    if (context is None) or (context == ""):
        prompt = f"Answer the user question: {question}"
    else:
        prompt = f"""
        Answer the user question: {question} based on the provided context.

        The context includes the following details for every movie:
        - "Title" of the movie
        - "Vote Average" - the average rating of the movie
        - "Budget" - the budget allocated for the movie in the US dollars
        - "Revenue" - the total revenue generated by the movie in the US dollars
        - "Release Date" - the date the movie was released

        Provide a detailed answer to the question, including relevant information from the context.
        Add information about the movie plot and characters from your internal knowledge base for every movie mentioned in the context.
        The information about each movie must be on a separate line.

        Context: 
        {context}
        """
    
    
    response = llm.invoke(prompt)
    return response

**4. Define Function Retreiving Context from Postgres**

In [153]:
def retrieve_context_from_postgres(years=5):
    db_params = {
    "host": "localhost",
    "port": 5432,
    "dbname": "postgres",
    "user": "postgres",
    "password": "password"
    }
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()

    query = """
    SELECT name, vote_average, budget, revenue, release_date
    FROM omdb.movies
    WHERE release_date >= NOW() - INTERVAL %s
    AND revenue > 0 AND budget > 0 AND vote_average > 0
    ORDER BY vote_average DESC, budget ASC
    LIMIT 3;
    """

    cursor.execute(query, (f'{years} years',))
    
    context = ""

    for row in cursor.fetchall():
        context += f"Movie title: {row[0]}, Vote Average: {row[1]}"
        context += f", Budget: {row[2]}, Revenue: {row[3]}, Release Date: {row[4]}\n"

    cursor.close()
    conn.close()

    return context

**6. Work With LLM Without Providing Context**

Requesting the LLM to answer based on its internal knowledge base. It will be hard for the LLM to come up with a proper answer based on the data it was trained on which can lead to hallucinations. Plus, it's highly likely the returned movies will have release years that don't satisfy the search criteria.

In [ ]:
years = 5

question = f"What are the top 3 movies released in the last {years} years with the highest vote average and the lowest budget?"

answer = answer_question(question, None)

print("LLM's answer without context:")
print(answer)

**7. Augment LLM With Context From Postgres**

Using the RAG approach to provide the LLM with more details stored in Postgres. This will augment the LLM's behavior, letting it answer more accurately and with no or fewer hallucinations. You can minimize the hallucinations by tweaking the LLM parameters or the prompt.

In [ ]:
years = 5

question = f"What are the top 3 movies released in the last {years} years with the highest vote average and the lowest budget?"

context = retrieve_context_from_postgres(years)

print("Context from Postgres:")
print(context)

answer = answer_question(question, context)

print("LLM's answer:")
print(answer)